In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from detectors.IKS import IKS

D:\Anaconda\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
path = "C:/Users/luiz_/Projects/DataStreams/DataStreams-Quantification/datasets/test"
files = os.listdir(path)
datasets = [file.split(".")[0] for file in files]
files = [pd.read_csv(f"{path}/{f}") for f in files]
print(datasets)

['AedesQuinx', 'AedesSex', 'Arabic-digit', 'wine']


In [3]:
print(files)

[             wbf      eh_1      eh_2      eh_3      eh_4      eh_5      eh_6  \
0     474.193548  0.178585  0.172441  0.043769  0.265499  0.068176  0.128343   
1     506.896552  0.218450  0.218481  0.027408  0.432591  0.176818  0.229486   
2     565.384615  0.034858  0.058285  0.173607  0.036100  0.377893  0.048056   
3     558.227848  0.070814  0.146803  0.364292  0.064749  0.678501  0.100370   
4     558.227848  0.109654  0.195359  0.340204  0.092397  0.624805  0.127890   
...          ...       ...       ...       ...       ...       ...       ...   
3995  436.633663  0.351219  0.316821  0.280244  0.291517  0.355871  0.153607   
3996  436.633663  1.389803  1.347412  0.671559  1.361457  0.613862  0.397510   
3997  501.136364  0.370440  0.379923  0.139380  0.993764  0.257972  0.697455   
3998  495.505618  0.495138  0.495532  0.056246  1.565005  0.193768  1.099718   
3999  506.896552  0.226639  0.228782  0.062082  0.318494  0.206956  0.128963   

          eh_7      eh_8      eh_9  ..

In [4]:
clf = RandomForestClassifier(n_estimators=200)

In [5]:
table = pd.DataFrame(index=[datasets])
window_size = 300 #window parameter to build the images for comparison
table

""
AedesQuinx
AedesSex
Arabic-digit
wine


In [6]:
for i, stream in enumerate(files):
    contexts = stream.iloc[:, -1]
    stream = stream.iloc[:, :-1]
    stream.iloc[:, -1].replace(2, int(0), inplace=True)

    print(f"dataset: {datasets[i]}")

    threshold = 1.95
    
    iks = IKS(stream, 1500, window_size, clf, contexts, threshold)
    iks.runslidingwindow()
     
    

dataset: AedesQuinx
classle 10/2500
1    1.0
Name: proportion, dtype: float64
    IKS  CC  ACC  MS  DyS  real
0     0   0    0   0    0     1
1     1   1    1   1    1     1
2     0   0    0   0    0     1
3     0   0    0   0    0     1
4     0   0    0   0    0     1
5     0   0    0   0    0     1
6     1   1    1   1    1     1
7     0   0    0   0    0     1
8     0   0    0   0    0     1
9     0   0    0   0    0     1
10    0   1    1   1    1     1
classle 11/2500
1    1.0
Name: proportion, dtype: float64
    IKS  CC  ACC  MS  DyS  real
0     0   0    0   0    0     1
1     1   1    1   1    1     1
2     0   0    0   0    0     1
3     0   0    0   0    0     1
4     0   0    0   0    0     1
5     0   0    0   0    0     1
6     1   1    1   1    1     1
7     0   0    0   0    0     1
8     0   0    0   0    0     1
9     0   0    0   0    0     1
10    0   1    1   1    1     1
11    0   1    1   1    1     1
classle 12/2500
1    1.0
Name: proportion, dtype: float64
    IK

Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\luiz_\AppData\Local\Temp\ipykernel_114460\94923575.py", line 11, in <module>
    iks.runslidingwindow()
  File "C:\Users\luiz_\Projects\DataStreams\DataStreams-Quantification\detectors\IKS.py", line 21, in runslidingwindow
    self.apply_qtf(new_instance)
  File "C:\Users\luiz_\Projects\DataStreams\DataStreams-Quantification\interface_class\DriftDetector.py", line 39, in apply_qtf
  File "C:\Users\luiz_\Projects\DataStreams\DataStreams-Quantification\quantifiers\ApplyQtfs.py", line 113, in aplly_qtf
    pred_pos_prop = self.apply_quantifier(quantifier=qtf, clf=self.model,
  File "C:\Users\luiz_\Projects\DataStreams\DataStreams-Quantification\quantifiers\ApplyQtfs.py", line 40, in apply_quantifier
    acc.fit(train[0], train[1])
  File "C:\Users\luiz_\Projects\DataStreams\DataStreams-Quan